In [ ]:

import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import os
from glob import glob
import cv2

from keras.models import Sequential 
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img


import warnings
warnings.filterwarnings('ignore')

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [2]:
train_dir = '/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/train'
test_dir =  '/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/test'
val_dir = '/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/val'

In [3]:

def picture_separation(folder):
    y = []
    x = []
    image_list = []

    for foldername in os.listdir(folder):
        if not foldername.startswith('.'):
            if foldername == "NORMAL":
                label = 0
            elif foldername == "PNEUMONIA":
                label = 1
            else:
                label = 2
                
            for image_filename in os.listdir(folder + "/"+ foldername):
                img_file = cv2.imread(folder + "/" + foldername + '/' + image_filename,0)               
                

                if img_file is not None:
                    img = cv2.resize(img_file,(64,64))
                    img_arr = img_to_array(img) / 255
                    x.append(img_arr)
                    y.append(label)
                    image_list.append(foldername + '/' + image_filename)
                                        
    X = np.asarray(x)
    y = np.asarray(y)
    
    

    return X,y,image_list

          
        

In [4]:
X_train, y_train, img_train = picture_separation(train_dir)

train_df = pd.DataFrame(img_train, columns=["images"])
train_df["target"] = y_train


In [ ]:
train_df.head()

In [ ]:
train_df.info()

In [ ]:
sns.countplot(train_df["target"])
plt.title("NORMAL/PNOMONİA")
plt.show()
print(train_df["target"].value_counts())

In [8]:
X_val, y_val, img_val = picture_separation(val_dir)

val_df = pd.DataFrame(img_val, columns=["images"])
val_df["target"] = y_val

In [ ]:
val_df.head()

In [ ]:
sns.countplot(val_df["target"])
plt.title("NORMAL/PNOMONİA")
plt.show()
print(val_df["target"].value_counts())

In [11]:
X_test, y_test, img_test = picture_separation(test_dir)

test_df = pd.DataFrame(img_test, columns=["images"])
test_df["target"] = y_test

In [ ]:
test_df.tail()

In [ ]:
sns.countplot(test_df["target"])
plt.title("NORMAL/PNOMONİA")
plt.show()
print(test_df["target"].value_counts())

In [14]:
 full_data = pd.concat([train_df, test_df, val_df], axis=0, ignore_index=True)

In [ ]:
print(full_data.head())
print(full_data.tail())

In [ ]:
plt.figure(figsize=(12,8))

img = load_img(train_dir + "/" + full_data["images"][3875])
plt.imshow(img)
plt.title("NORMAL", color = "green", size = 14)
plt.grid(color='#CCCCCC', linestyle='--')
plt.show()

In [ ]:
plt.figure(figsize=(10,7))

img = load_img(train_dir + "/" + full_data["images"][0])
plt.imshow(img)
plt.title("PNEUMONIA", color = "green", size = 14)
plt.grid(color='#CCCCCC', linestyle='--')
plt.show()

In [ ]:
sns.countplot(full_data["target"])
plt.title("NORMAL/PNOMONİA")
plt.show()
print(full_data["target"].value_counts())


In [ ]:
plt.figure(figsize=(15,8))

plt.subplot(2,3,1) 
img = load_img(train_dir + "/" + full_data["images"][0])
plt.imshow(img)
plt.title("PNEUMONIA", color = "blue", size = 14)
plt.axis("off")


plt.subplot(2,3,2) 
img = load_img(train_dir + "/" + full_data["images"][1])
plt.imshow(img)
plt.title("PNEUMONIA", color = "blue", size = 14)
plt.axis("off")

plt.subplot(2,3,3) 
img = load_img(train_dir + "/" + full_data["images"][10])
plt.imshow(img)
plt.title("PNEUMONIA", color = "blue", size = 14)
plt.axis("off")

plt.subplot(2,3,4) 
img = load_img(train_dir + "/" + full_data["images"][3875])
plt.imshow(img)
plt.title("NORMAL", color = "green", size = 14)
plt.axis("off")

plt.subplot(2,3,5) 
img = load_img(train_dir + "/" + full_data["images"][3876])
plt.imshow(img)
plt.title("NORMAL", color = "green", size = 14)
plt.axis("off")

plt.subplot(2,3,6) 
img = load_img(train_dir + "/" + full_data["images"][3877])
plt.imshow(img)
plt.title("NORMAL", color = "green", size = 14)
plt.axis("off")

plt.suptitle("NORMAL or PNEUMONIA", size = 16, color = "darkred")
plt.show()

In [20]:
# the shape
X_train.shape

(5216, 64, 64, 1)

In [ ]:
className = glob(train_dir + '/*' )
numberOfClass = len(className)
print("NumberOfClass: ",numberOfClass)

In [22]:
batch_size = 32

In [23]:
train_datagen = ImageDataGenerator(rescale= 1./255,
                   shear_range = 0.3,
                   horizontal_flip=True,
                   zoom_range = 0.3)

In [24]:
test_datagen = ImageDataGenerator(rescale= 1./255)

In [25]:
val_datagen = ImageDataGenerator(rescale= 1./255)

In [26]:
train_generator = train_datagen.flow_from_directory(
        train_dir, 
        target_size=(64, 64),
        batch_size = batch_size,
        color_mode = "grayscale",
        class_mode= "binary")



Found 5216 images belonging to 2 classes.


In [27]:
test_generator = test_datagen.flow_from_directory(
        test_dir, 
        target_size=(64, 64),
        batch_size = batch_size,
        color_mode = "grayscale",
        class_mode= "binary")

Found 624 images belonging to 2 classes.


In [28]:
val_generator = test_datagen.flow_from_directory(
        val_dir, 
        target_size=(64, 64),
        batch_size = batch_size,
        color_mode = "grayscale",
        class_mode= "binary")

Found 16 images belonging to 2 classes.


In [29]:
X_train.shape[1:]

(64, 64, 1)

In [30]:
model = Sequential()
model.add(Conv2D(32,(3,3),input_shape = X_train.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D())

model.add(Conv2D(32,(3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D())

model.add(Conv2D(64,(3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D())

model.add(Flatten())
model.add(Dense(1024))
model.add(Activation("relu"))
model.add(Dropout(0.4))
model.add(Dense(1)) # output
model.add(Activation("sigmoid"))

model.compile(loss = "binary_crossentropy",
              optimizer = "rmsprop",
              metrics = ["accuracy"])


<ul>
    <li style = "color:green" > <p style = "color:black" >When our model reaches the desired level, we use early stopping to stop it.</p> </li>
</ul>

In [31]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=2)

<ul>
    <li style = "color:green" > <p style = "color:black" >
We train our model that we created.</p> </li>
</ul>

In [ ]:
history = model.fit_generator(
                    train_generator,
                    steps_per_epoch=5216//32,
                    epochs=20,
                    validation_data=test_generator,
                    validation_steps=624//32,
                    callbacks=[early_stopping])

In [33]:
print("Accuracy of the model is - " , model.evaluate_generator(test_generator)[1]*100 , "%")
print("Loss of the model is - " , model.evaluate_generator(test_generator)[0])

Accuracy of the model is -  90.54487347602844 %
Loss of the model is -  0.2816219925880432


In [34]:
#predictions = model.predict_generator(test_generator)

In [35]:
history.history.keys()

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])

In [ ]:
plt.figure()
plt.plot(history.history["loss"],label = "Train Loss")
plt.plot(history.history["val_loss"],label = "Validation Loss")
plt.legend()
plt.show()

In [ ]:
plt.figure()
plt.plot(history.history["accuracy"],label = "Train Accuracy")
plt.plot(history.history["val_accuracy"],label = "Validation Accuracy")
plt.legend()
plt.show()